In this notebook, we will use the technique of [ensembling](https://en.wikipedia.org/wiki/Ensemble_learning) to solve the [Spaceship Titanic](https://www.kaggle.com/competitions/spaceship-titanic) competition. Additionally, we will leverage the popular hyperparameter tuning package [Optuna](https://github.com/optuna/optuna) to tune our models.

The objective is to use ensembling in the form of [stacking](https://en.wikipedia.org/wiki/Ensemble_learning#Stacking), with the following models making up the stack:
* [Logistic Regression](https://en.wikipedia.org/wiki/Logistic_regression)
* [AdaBoost](https://en.wikipedia.org/wiki/AdaBoost) with [Decision Tree](https://en.wikipedia.org/wiki/Decision_tree_learning) as its base classifier
* [Random Forest](https://en.wikipedia.org/wiki/Random_forest)
* [XGBoost](https://en.wikipedia.org/wiki/XGBoost)
* [LightGBM](https://en.wikipedia.org/wiki/LightGBM)

This is Part 3 of a three part series:
* Part 1: [Spaceship Titanic - Exploratory Data Analysis](https://www.kaggle.com/code/defcodeking/spaceship-titanic-exploratory-data-analysis)
* Part 2: [Spaceship Titanic - Logistic Regression Baselines](https://www.kaggle.com/code/defcodeking/spaceship-titanic-logistic-regression-baselines)
* Part 3: [Ensembling (And Optuna 😉) Is All You Need!](https://www.kaggle.com/code/defcodeking/ensembling-and-optuna-is-all-you-need) (you are here!)

> Note: For a quickstart on Optuna, check out [Tutorial — Optuna](https://optuna.readthedocs.io/en/stable/tutorial/index.html).

# Imports

In [ ]:
import functools
import os
import random

import lightgbm as lgb
import numpy as np
import optuna
import pandas as pd
import xgboost as xgb
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

import warnings
warnings.filterwarnings("ignore")

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
seed_everything()

# Config

We will define a global `Config` class which holds common configuration.

We will also define a configuration class for each of the models that will be used in the ensemble. These classes will hold the default hyperparameters, static parameters (parameters that are not tuned), extra parameters to pass to `.fit()` etc., for the respective models.

Additionally, we will have a mapping between model names and their configuration classes so that we can get to them easily.

> Note: The `get_fit_params()` method is required in the configuration classes of the models since there are subtle differences between what the `.fit()` method of each model accepts. For example, `LogisticRegression.fit()` has no parameter called `eval_set` while `XGBClassifier.fit()` does. `LogisticRegression` takes the `verbose` argument in the initializer while `XGBoost` takes it in `.fit()`. `XGBoost`, since version 1.6.0, has moved arguments like `callbacks` and `eval_metric` to the initializer while `LightGBM` continues to use them in `.fit()`. These differences are reconciled by dividing the arguments appropriately between `STATIC_PARAMS` and the output of `get_fit_params()`. 

In [ ]:
class Config:
    DATA_DIR = "../input/spaceship-titanic-prepared-datasets"
    L1_N_TRIALS = 100
    L2_N_TRIALS = 20
    N_JOBS = 2
    
    # Map internal identifier to human-friendly name
    MODELS = {
        "lr": "Logisitc Regression",
        "ada": "AdaBoost",
        "rf": "Random Forest",
        "xgb": "XGBoost",
        "lgb": "LightGBM",
    }
    
    @classmethod
    def filepath(cls, filename):
        return os.path.join(cls.DATA_DIR, filename)

In [ ]:
# Configuration for Logistic Regression
class LRConfig:
    DEFAULT_VALUES = {
        "tol": 1e-4,
        "C": 1.0,
        "solver": "lbfgs",
    }
    STATIC_PARAMS = {
        "max_iter": 1000,
        "verbose": False,
    }
    
    USE_PRUNER = False
    
    @classmethod
    def get_fit_params(cls, X_train, y_train, X_val, y_val, params):
        return {}

In [ ]:
# Configuration for AdaBoost
class AdaConfig:
    DEFAULT_VALUES = {
        "base_estimator": None,
        "n_estimators": 50,
        "learning_rate": 1.0,
        
    }
    STATIC_PARAMS = {}
    
    USE_PRUNER = False
    
    @classmethod
    def get_fit_params(cls, X_train, y_train, X_val, y_val, params):
        return {}

In [ ]:
# Configuration for Random Forest
class RFConfig:
    DEFAULT_VALUES = {
        "n_estimators": 100,
        "max_depth": None,
        "min_samples_split": 2,
        "min_samples_leaf": 1,
        "max_features": "sqrt",
        "bootstrap": True,
        "ccp_alpha": 0.0,
        "max_samples": None,
    }

    STATIC_PARAMS = {
        "n_jobs": Config.N_JOBS,
    }
    
    USE_PRUNER = False
    
    @classmethod
    def get_fit_params(cls, X_train, y_train, X_val, y_val, params):
        return {}

In [ ]:
# Configuration for XGBoost
class XGBConfig:
    EVAL_METRIC = "logloss"
    
    DEFAULT_VALUES = {
        "max_depth": 6,
        "n_estimators": 100,
        "alpha": 0.0,
        "lambda": 1.0,
        "learning_rate": 0.3,
        "colsample_bytree": 1.0,
        "colsample_bylevel": 1.0,
        "min_child_weight": 1.0,
        "sampling_method": "uniform",
        "early_stopping_rounds": None,
    }
    
    STATIC_PARAMS = {
        "tree_method": "gpu_hist",
        "use_label_encoder": False,
        "n_jobs": Config.N_JOBS,
        "predictor": "gpu_predictor",
        "max_bin": 1024,
        "eval_metric": EVAL_METRIC,
    }
    
    USE_PRUNER = True
    
    @classmethod
    def get_fit_params(cls, X_train, y_train, X_val, y_val, params):
        return {
            "eval_set": [(X_train, y_train), (X_val, y_val)],
            "verbose": False,
        }

In [ ]:
# Configuration for LightGBM
class LGBConfig:
    DEFAULT_VALUES = {
        "num_leaves": 31,
        "max_depth": -1,
        "learning_rate": 0.1,
        "n_estimators": 100,
        "reg_alpha": 0.0,
        "reg_lambda": 0.0,
        "min_child_samples": 20,
        "subsample_for_bin": 200000,
    }
    
    STATIC_PARAMS = {
        "n_jobs": Config.N_JOBS,
        "verbose": -1,
        "objective": "binary",
    }
    
    USE_PRUNER = True
    
    @classmethod
    def get_fit_params(cls, X_train, y_train, X_val, y_val, params):
        # To suppress training output
        callbacks = params.get("callbacks", []) + [lgb.log_evaluation(period=0)]
        return {
            "eval_set": [(X_train, y_train), (X_val, y_val)],
            "eval_metric": "logloss",
            "callbacks": callbacks,
        }

In [ ]:
CONFIG_MAP = {
    "lr": LRConfig,
    "ada": AdaConfig,
    "rf": RFConfig,
    "xgb": XGBConfig,
    "lgb": LGBConfig,
}

# Load Datasets

> Note: This notebook uses the datasets as prepared in [Spaceship Titanic - Logistic Regression Baselines](https://www.kaggle.com/code/defcodeking/spaceship-titanic-logistic-regression-baselines). Link to dataset: [Spaceship Titanic Prepared Datasets](https://www.kaggle.com/datasets/defcodeking/spaceship-titanic-prepared-datasets).

> Note: While the experiments in Part 2 of this series suggest that using only `GroupId` (extracted from `PassengerId`) with label encoding gives the best model, we will use both `CabinNum` and `GroupId` label encoded since the former beats the latter only marginally (~0.23%).

We will load the datasets.

In [ ]:
train_df = pd.read_csv(Config.filepath("train_prepared_both_le.csv"))
test_df = pd.read_csv(Config.filepath("test_prepared_both_le.csv"))

In [ ]:
train_df.head()

In [ ]:
test_df.head()

# Model Map

The `MODEL_MAP` variable will be used as a mapping from the string identifier of a model to its associated class.

In [ ]:
MODEL_MAP = {
    "lr": LogisticRegression,
    "ada": AdaBoostClassifier,
    "rf": RandomForestClassifier,
    "xgb": xgb.XGBClassifier,
    "lgb": lgb.LGBMClassifier,
}

# Training Loop

The `train()` function will be used as a generic training loop. It takes the training and test dataframes, a string identifier for the model that is being trained, the hyperparameters for the model and an optional verbosity argument. In the end, it returns the predictions for the training set, the test set and the overall accuracy.

In [ ]:
def train(df, test_df, model, params, verbose=True):
    # Create copies so that original datatsets do not change
    df = df.copy()
    test = test_df.drop("PassengerId", axis=1)
    
    df["preds"] = pd.NA
    
    drop = ["Transported", "preds", "kfold"]
    
    # Get the initializer class and the configuration class
    klass = MODEL_MAP[model]
    config = CONFIG_MAP[model]
    
    # Default values in the config class are for tuned parameters
    # So, only those are filtered from params
    # This is mainly added for AdaBoost, since it has a slightly different objective
    params = {k: v for k, v in params.items() if k in config.DEFAULT_VALUES}
    
    # Add default values for parameters not defined in params
    params.update({k: v for k, v in config.DEFAULT_VALUES.items() if k not in params})
    
    # Add static params - Parameters that are not tuned
    params.update(config.STATIC_PARAMS)
    
    # For storing total accuracy across folds for averaging
    total_acc = 0.0
    
    # Empty list for storing test predictions in each fold
    test_preds = []
    
    for fold in range(5):
        train = df[df["kfold"] != fold]
        
        # Get training features and labels
        y_train = train["Transported"]
        X_train = train.drop(drop, axis=1)
        
        val = df[df["kfold"] == fold]
        
        # Get validation features and labels
        y_val = val["Transported"]
        X_val = val.drop(drop, axis=1)
        
        # Initialize model
        clf = klass(**params)
        
        # Get parameters for .fit() other than X and y
        fit_params = config.get_fit_params(
            X_train=X_train,
            y_train=y_train,
            X_val=X_val,
            y_val=y_val,
            params=params
        )
        
        # Train model on training set
        clf.fit(
            X=X_train,
            y=y_train,
            **fit_params,
        )
        
        # Make predictions on validation set
        val_pred = clf.predict(X_val)
        acc = metrics.accuracy_score(y_val, val_pred)
        
        # Report accuracy if verbose is True
        if verbose is True:
            print(f"\tFold {fold + 1} - Accuracy = {acc: .4f}")
        
        # Add to total accuracy
        total_acc += acc
        
        # Make predictions on validation set again
        # But this time in terms of probabilities
        # And store in the df
        # These will be used in the meta model
        df.loc[val.index, "preds"] = clf.predict_proba(X_val)[:, 1]
        
        # Get the test predictions for this fold in terms of probability
        test_preds.append(clf.predict_proba(test)[:, 1])
        
    acc = total_acc / 5
    
    if verbose is True:
        print(f"\tOverall accuracy = {acc: .4f}")   
    
    # Calculate final test predictions
    # These will be used in the meta model
    test_preds = np.vstack(test_preds)
    test_preds = test_preds.mean(axis=0)
    
    # Return val preds, test preds and overall accuracy
    return df["preds"].values, test_preds, acc

# Optuna Objectives

The functions below will act as the objective functions that should be used for each of the models. These take the Optuna trial, the training dataframe and the test dataframe as arguments. They first use Optuna to get a dictionary of parameters and then call the `train()` function with the appropriate arguments. 

In [ ]:
# Objective for Logistic Regression
def lr_objective(trial, train_df, test_df):
    params = {
        "tol": trial.suggest_float("tol", 1e-6, 1e-4, log=True),
        "C": trial.suggest_float("C", 0.5, 2.0, log=True),
        "solver": trial.suggest_categorical("solver", ["newton-cg", "lbfgs", "liblinear", "sag", "saga"]),
    }
    
    _, _, acc = train(df=train_df, test_df=test_df, model="lr", params=params, verbose=False)
    return acc

In [ ]:
# Objective for AdaBoost
def adaboost_objective(trial, train_df, test_df):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 500, log=True),
        "learning_rate": trial.suggest_float("learning_rate", 0.1, 5.0, log=True),
    }
    
    tune_estimator = trial.suggest_categorical("tune_estimator", [True, False])
    
    if tune_estimator:
        # Parameters for the Decision Tree in AdaBoost
        max_depth = trial.suggest_int("max_depth", 1, 50)
        min_samples_split = trial.suggest_int("min_samples_split", 2, 10, log=True)
        min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 5, log=True)
        ccp_alpha = trial.suggest_float("ccp_alpha", 0.01, 1.0, log=True)
        
        params["base_estimator"] = DecisionTreeClassifier(
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            ccp_alpha=ccp_alpha,
        )
    
    _, _, acc = train(df=train_df, test_df=test_df, model="ada", params=params, verbose=False)
    return acc

In [ ]:
# Objectuve for Random Forest
def rf_objective(trial, train_df, test_df):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 1, 500),
        "max_depth": trial.suggest_int("max_depth", 1, 50),
        "min_samples_split": trial.suggest_int("min_samples_plit", 2, 10, log=True),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 5, log=True),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None]),
        "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
        "ccp_alpha": trial.suggest_float("ccp_alpha", 0.01, 1.0, log=True),
    }
    
    if params["bootstrap"] is True:
        params["max_samples"] = trial.suggest_float("max_samples", 0.01, 1.0, log=True)
    
    _, _, acc = train(df=train_df, test_df=test_df, model="rf", params=params, verbose=False)
    return acc

In [ ]:
# Objective for XGBoost
def xgb_objective(trial, train_df, test_df):
    params = {
        "max_depth": trial.suggest_int("max_depth", 1, 11),
        "n_estimators": trial.suggest_int("n_estimators", 5, 500),
        "alpha": trial.suggest_uniform("alpha", 0.0, 5.0),
        "lambda": trial.suggest_float("lambda", 1.0, 5.0, log=True),
        "learning_rate": trial.suggest_float("learning_rate", 0.03, 0.8, log=True),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.2, 1.0),
        "min_child_weight": trial.suggest_uniform("min_child_weight", 1, 100),
        "sampling_method": trial.suggest_categorical("sampling_method", ["uniform", "gradient_based"]),
        "early_stopping_rounds": trial.suggest_int("early_stopping_rounds", 5, 20, step=5)
    }
    
    obs_k = f"validation_1-{XGBConfig.EVAL_METRIC}"
    params["callbacks"] = [optuna.integration.XGBoostPruningCallback(trial, obs_k)]
    
    _, _, acc = train(df=train_df, test_df=test_df, model="xgb", params=params, verbose=False)
    return acc

In [ ]:
# Objective for LightGBM
def lgb_objective(trial, train_df, test_df):
    params = {
        "num_leaves": trial.suggest_int("num_leaves", 31, 100, log=True),
        "max_depth": trial.suggest_int("max_depth", 1, 100, log=True),
        "learning_rate": trial.suggest_float("learning_rate", 1e-4, 1.0),
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 5.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 5.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 20, 50, log=True),
        "subsample_for_bin": trial.suggest_int("subsample_for_bin", 2000, 8000),
    }
    
    params["callbacks"] = [optuna.integration.LightGBMPruningCallback(trial, "logloss", "valid_1")]
    
    _, _, acc = train(df=train_df, test_df=test_df, model="lgb", params=params, verbose=False)
    return acc

In [ ]:
OBJECTIVE_MAP = {
    "lr": lr_objective,
    "ada": adaboost_objective,
    "rf": rf_objective,
    "xgb": xgb_objective,
    "lgb": lgb_objective,
}

# Hyperparameter Search

The function `hyperparameter_search()` finds the best hyperparameters for the given model by utilizing the proper objective function. It takes the training dataframe, test dataframe and the string identifier of the model for which hyperparameters are required.

In [ ]:
def hyperparameter_search(train_df, test_df, model, n_trials=Config.L1_N_TRIALS):
    # Turn off verbose output
    v = optuna.logging.get_verbosity()
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    
    objective = OBJECTIVE_MAP[model]
    objective = functools.partial(objective, train_df=train_df, test_df=test_df)
    
    # Check if pruning is required
    pruner = optuna.pruners.HyperbandPruner() if CONFIG_MAP[model].USE_PRUNER is True else None
    
    sampler = optuna.samplers.TPESampler(seed=42)
    study = optuna.create_study(
        direction="maximize",
        pruner=pruner,
        sampler=sampler,
    )
    
    study.optimize(objective, n_trials=n_trials)
    
    # Restore verbosity level
    optuna.logging.set_verbosity(v)

    return study.best_params

# Datasets for LGBMClassifier

LightGBM is unique from other boosting algorithms in that it supports categorical features out of the box. We only need to make sure they are label encoded and have their datatype as `category`. The function below takes the two datasets and label encodes all the one-hot encoded columns.

In [ ]:
def lgb_datasets(train_df, test_df):
    # Make copies so that original datasets remain unchanged
    train_df = train_df.copy()
    test_df = test_df.copy()
    
    # Drop Transported and kfold
    drop = ["Transported", "kfold"]
    dropped = train_df[drop].values
    train_df = train_df.drop(drop, axis=1)
    
    # Drop PassengerId
    passenger_id = test_df["PassengerId"].values
    test_df = test_df.drop("PassengerId", axis=1)
    
    # Add suffix to index and store indices
    # So that the dataframes can be merged and split
    train_df = train_df.rename("train_{}".format)
    test_df = test_df.rename("test_{}".format)
    
    tr_idx = train_df.index
    te_idx = test_df.index
    
    # Merge
    df = pd.concat([train_df, test_df])
    
    oh_cols = ["CabinDeck", "HomePlanet", "Destination", "GroupSize"]
    
    for oh_col in oh_cols:
        # Get all columns associated with the one-hot column
        columns = [column for column in df.columns if column.startswith(f"{oh_col}_")]
        
        # .idxmax() returns that column name which has the maximum value in the row
        values = df[columns].idxmax(axis=1)
        
        # Get all levels and make a mapping from level to index
        levels = values.value_counts().index
        mapping = {level: idx for idx, level in enumerate(levels)}
        
        # Add column with the mapping and specify type as category
        df[oh_col] = values.map(mapping).astype("category")
        
        # Drop one-hot columns
        df = df.drop(columns, axis=1)
        
    # Make sure other categorical features have the correct type
    missing = (col for col in df.columns if col.endswith("_missing"))
    others = ["CryoSleep", "VIP", "Alone", "CabinNum", "GroupId", *missing]
    df[others] = df[others].astype("category")
        
    # Split and add dropped columns
    train_df = df.loc[tr_idx, :]
    train_df[drop] = dropped
    
    test_df = df.loc[te_idx, :]
    test_df["PassengerId"] = passenger_id
    
    return train_df, test_df

## Example

In [ ]:
tr, te = lgb_datasets(train_df, test_df)
tr.columns, te.columns

In [ ]:
tr.head()

# Ensemble Class

The `Ensemble` class will implement all the logic required for stacking. It takes the training dataframe, the test dataframe and an optional list of strings which specifies the models that should be excluded from the ensemble. It has the following methods:

- `fit_level_one_models()`: This method fits all the different models (Logistic Regression, XGBoost, etc.) on the original training data and also creates the training set and test sets for the meta-classifier that will generate the final predictions.
- `fit_level_two_model()`: This method fits a logistic regression model on the dataset generated by `fit_level_one_models()` and gets the final predictions.

In [ ]:
class Ensemble:
    def __init__(self, train_df, test_df, exclude=None):
        self.train_df = train_df
        self.test_df = test_df
        
        models = Config.MODELS.keys()
        
        # Exclude models
        if exclude is not None:
            models = models - exclude
            
        self.models = list(models)
        
        # Create empty dataframe that will store
        # The training set for the level 2 model
        columns = [f"{m}_preds" for m in self.models]
        extra_cols = ["Transported", "kfold"]
        
        meta_train_df = pd.DataFrame(columns=columns + extra_cols)
        meta_train_df[extra_cols] = train_df[extra_cols]
        
        self.meta_train_df = meta_train_df
        
        # Create empty dataframe that will store
        # The test set for the level 2 model
        meta_test_df = pd.DataFrame(columns=["PassengerId"] + columns)
        meta_test_df["PassengerId"] = test_df["PassengerId"]
        
        self.meta_test_df = meta_test_df
        
    def fit_level_one_models(self):
        print("Training level 1 models...")
        
        for model in self.models:
            if model == "lgb":
                # Modify dataset for LGBMClassifier
                train_df, test_df = lgb_datasets(self.train_df, self.test_df)
            else:
                train_df, test_df = self.train_df, self.test_df
            
            print(f"{Config.MODELS[model]}:")
            
            print("\tFinding optimal hyperparameters using Optuna...")
            params = hyperparameter_search(
                train_df=train_df, test_df=test_df, model=model
            )
            
            print(f"\n\tBest params: {params}\n")
            
            print("\tTraining model with optimal parameters...\n")
            val_preds, test_preds, acc = train(
                df=train_df,
                test_df=test_df,
                model=model,
                params=params
            )
            
            print("\tDone!\n")
            
            # Add predictions to the datasets for the level 2 model
            self.meta_train_df[f"{model}_preds"] = val_preds
            self.meta_test_df[f"{model}_preds"] = test_preds
            
    def fit_level_two_model(self):    
        print("Training a Logistic Regression model as level 2 model...")
        
        train_df = self.meta_train_df
        test_df = self.meta_test_df
        
        print("\tFinding optimal hyperparameters using Optuna...")
        params = hyperparameter_search(
            train_df=train_df,
            test_df=test_df,
            model="lr",
            n_trials=Config.L2_N_TRIALS,
        )

        print(f"\n\tBest params: {params}\n")

        print("\tTraining model with optimal parameters...\n")
        
        _, test_preds, _ = train(
            df=train_df,
            test_df=test_df,
            model="lr",
            params=params
        )
        
        print("\tDone!")
        
        self.meta_test_df["Transported"] = test_preds >= 0.5
        
        return self.meta_test_df

# Training

Finally, we will use the `Ensemble` class to train our ensemble model.

In [ ]:
# Initialize the ensemble
ensemble = Ensemble(train_df, test_df)
ensemble.models

In [ ]:
# Fit the level one models
ensemble.fit_level_one_models()

In [ ]:
# Generated training set for level 2 model
ensemble.meta_train_df.head()

In [ ]:
# Generated test set for level 2 model
ensemble.meta_test_df.head()

In [ ]:
# Fit the level 2 model
test_predictions = ensemble.fit_level_two_model()

In [ ]:
test_predictions.head()

In [ ]:
test_predictions["Transported"].value_counts()

# Submission

Now we will generate the submissions file.

In [ ]:
submission = test_predictions[["PassengerId", "Transported"]]
submission.to_csv("submission.csv", index=False)

# Conclusion

In this notebook, we used the technique of ensembling in the form of stacking coupled with Optuna to solve the Spaceship Titanic competition.

Thank you for reading!